In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import requests
import pandas as pd
import asyncio
import aiohttp

In [11]:
def job_offers_wtj(
        job_title: str = "data analyst",
        pages: int = 1
):
    """
    ---
    Web scraping WTJ
    ---
    Lance un navigateur et créée un dataframe contenant les informations de
    chaque offre d'emploi pour un nombre de pages définis sur le site
    Welcome To The Jungle.
    ---
    Paramètres :
    - job_title: str : Le nom du métier pour lequel rechercher des offres.
    - pages: int : Le nombre de pages sur lesquels rechercher les offres.
    ---
    Retourne :
    - Le df contenant les informations de toute les offres récupérées.
    """
    # Instanciation de la liste contenant les liens pour les requêtes APIs.
    api_links = []
    # Lien de l'API de Welcome To The Jungle pour récupérer les données.
    api_link = f"https://api.welcometothejungle.com/api/v1/organizations"
    job = job_title.lower().replace(" ", "+")
    # Instanciation du driver Firefox.
    driver = webdriver.Firefox()
    # Instanciation du dataframe final.
    full_df = pd.DataFrame()
    # Nom des colonnes à garder dans le dataframe final.
    cols_to_keep = [
        "name",
        "salary_period",
        "experience_level",
        "apply_url",
        "contract_duration_min",
        "office.city",
        "office.address",
        "office.district",
        "office.latitude",
        "office.longitude",
        "office.zip_code",
        "profession.category.fr",
        "profession.name.fr"
        "name",
        "education_level",
        "application_fields.mode",
        "application_fields.name",
        "description",
        "organization.average_age",
        "organization.creation_year",
        "organization.default_language",
        "organization.description",
        "organization.industry",
        "organization.nb_employee",
        "contract_type",
        "salary_min",
        "salary_max",
        "education_level",
        "remote"
    ]
    try:
        for i in range(1, pages+1):
            url = f"https://www.welcometothejungle.com/fr/jobs?refinementList%5Boffices.country_code%5D%5B%5D=FR&query={job}&page={i}"
            # Ouvre chaque page sur le navigateur.
            driver.get(url)
            try:
                # Récupère le lien de chaque offre d'emploi sur la page.
                contents = WebDriverWait(driver, 20).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".sc-6i2fyx-0.gIvJqh"))
                )
                for content in contents:
                    link = content.get_attribute("href")
                    end_link = re.findall(r"/companies(.+)", link)[0]
                    full_link = api_link + end_link
                    # Rajoute le lien de chaque offre à la liste.
                    api_links.append(full_link)
            except Exception as e:
                print(f"Error scraping page {i} : {e}")
    finally:
        driver.quit()
    # Pour chaque lien de la liste, fait une requête API et stocke les informations dans un dataframe.
    for link_ in api_links:
        r = requests.get(link_)
        df = pd.json_normalize(
            r.json()["job"]
        )
        full_df = pd.concat([full_df, df], ignore_index=True)
    # Instanciation de la liste des colonnes à drop.
    cols_to_drop = [col for col in full_df.columns if col not in cols_to_keep]
    df = full_df.drop(columns=cols_to_drop)
    return df

In [12]:
df = job_offers_wtj("data analyst", 1)

In [6]:
from bs4 import BeautifulSoup
import pandas as pd
def clean_html(text):
    soup = BeautifulSoup(text, "html.parser")
    cleaned_text = soup.get_text(separator=" ")
    cleaned_text = cleaned_text.replace("\xa0", " ")
    return cleaned_text

df["description"] = df["description"].apply(clean_html)
df["organization.description"] = df["organization.description"].apply(clean_html)

In [7]:
df.to_csv("WTT_offers.csv")